In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

file_path_01 = '/content/drive/My Drive/amazon_cells_labelled.txt'
file_path_02 = '/content/drive/My Drive/yelp_labelled.txt'

df_01 = pd.read_csv(file_path_01, sep='\t', header=None, names=['Text', 'Label'])
df_02 = pd.read_csv(file_path_02, sep='\t', header=None, names=['Text', 'Label'])
df_sentenses = pd.concat([df_01, df_02], ignore_index=True)

df_sentenses.head()

,Text,Label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
from keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(df_sentenses.Text)
text_matrix = t.texts_to_sequences(df_sentenses.Text)

vocab_length = len(t.word_index)
print("vocab_length:", vocab_length)

vocab_length: 3258


In [ ]:
len_mat=[]
for i in range(len(text_matrix)):
    len_mat.append(len(text_matrix[i]))
features = max(len_mat)

In [ ]:
max(len_mat) #######################

32

In [ ]:
from keras.preprocessing.sequence import pad_sequences
text_pad = pad_sequences(text_matrix, maxlen=features, padding='post')

In [ ]:
text_pad.shape

(2000, 32)

In [ ]:
!pip install tensorflow keras

In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

features = max(len_mat)
vocab_length = len(t.word_index)

In [ ]:
x1 = Embedding(input_dim=vocab_length + 1, output_dim=32, input_length=features, embeddings_regularizer=keras.regularizers.l2(0.001))(inputs1)

In [ ]:
x1

<KerasTensor: shape=(None, 32, 32) dtype=float32 (created by layer 'embedding_1')>

In [ ]:
inputs1 = Input(shape=(features,))

x1 = Embedding(input_dim=vocab_length + 1, output_dim=32, input_length=features, embeddings_regularizer=keras.regularizers.l2(0.001))(inputs1)

x1 = LSTM(100, dropout=0.3, recurrent_dropout=0.2)(x1)

outputs1 = Dense(1, activation='sigmoid')(x1)

model1 = Model(inputs1, outputs1)

model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32)]              0         
                                                                 
 embedding (Embedding)       (None, 32, 32)            104288    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 157589 (615.58 KB)
Trainable params: 157589 (615.58 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model1.fit(x=text_pad,y=df_sentenses.Label,batch_size=100,epochs=10,verbose=1,shuffle=True,validation_split=0.2)

Epoch 1/10
16/16 [==============================] - 2s 96ms/step - loss: 0.0444 - acc: 0.9919 - val_loss: 1.0303 - val_acc: 0.7900
Epoch 2/10
16/16 [==============================] - 2s 95ms/step - loss: 0.0375 - acc: 0.9962 - val_loss: 1.1149 - val_acc: 0.7800
Epoch 3/10
16/16 [==============================] - 1s 86ms/step - loss: 0.0409 - acc: 0.9937 - val_loss: 1.0625 - val_acc: 0.7850
Epoch 4/10
16/16 [==============================] - 1s 90ms/step - loss: 0.0400 - acc: 0.9950 - val_loss: 0.9127 - val_acc: 0.7925
Epoch 5/10
16/16 [==============================] - 2s 141ms/step - loss: 0.0309 - acc: 0.9969 - val_loss: 0.9801 - val_acc: 0.8025
Epoch 6/10
16/16 [==============================] - 2s 152ms/step - loss: 0.0284 - acc: 0.9987 - val_loss: 1.1715 - val_acc: 0.7950
Epoch 7/10
16/16 [==============================] - 2s 149ms/step - loss: 0.0288 - acc: 0.9975 - val_loss: 1.2758 - val_acc: 0.7900
Epoch 8/10
16/16 [==============================] - 1s 89ms/step - loss: 0.0312 

In [ ]:
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
no_of_neurons = 100

inputs = Input((features,))

x = Embedding(input_dim=vocab_length + 1, output_dim=32, input_length=features, embeddings_regularizer=keras.regularizers.l2(0.001))(inputs)

att_in = LSTM(no_of_neurons, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)(x)

att_out = attention()(att_in)

outputs = Dense(1, activation='sigmoid', trainable=True)(att_out)

model = Model(inputs, outputs)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 32, 32)            104288    
                                                                 
 lstm_3 (LSTM)               (None, 32, 100)           53200     
                                                                 
 attention_1 (attention)     (None, 100)               132       
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 157721 (616.10 KB)
Trainable params: 157721 (616.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x=text_pad,y=df_sentenses.Label,batch_size=100,epochs=10,verbose=1,shuffle=True,validation_split=0.2)

Epoch 1/10
16/16 [==============================] - 5s 141ms/step - loss: 0.1408 - acc: 0.9494 - val_loss: 0.5249 - val_acc: 0.7825
Epoch 2/10
16/16 [==============================] - 1s 88ms/step - loss: 0.1425 - acc: 0.9625 - val_loss: 1.0110 - val_acc: 0.7850
Epoch 3/10
16/16 [==============================] - 1s 90ms/step - loss: 0.0636 - acc: 0.9775 - val_loss: 0.7291 - val_acc: 0.7850
Epoch 4/10
16/16 [==============================] - 2s 94ms/step - loss: 0.0334 - acc: 0.9950 - val_loss: 0.9664 - val_acc: 0.7875
Epoch 5/10
16/16 [==============================] - 1s 90ms/step - loss: 0.0305 - acc: 0.9925 - val_loss: 0.7972 - val_acc: 0.8025
Epoch 6/10
16/16 [==============================] - 2s 149ms/step - loss: 0.0256 - acc: 0.9937 - val_loss: 0.8817 - val_acc: 0.7975
Epoch 7/10
16/16 [==============================] - 2s 156ms/step - loss: 0.0210 - acc: 0.9956 - val_loss: 0.8738 - val_acc: 0.8025
Epoch 8/10
16/16 [==============================] - 2s 131ms/step - loss: 0.0167